# How to Extend Pandas DataFrames with Custom Methods to Supercharge Code Functionality and Readability

## A step-by-step guide to extending pandas DataFrames with custom methods including full examples of how to implement extensions for conditional probability and expected values

![Alt text](markus-spiske-hvSr_CVecVI-unsplash-1.jpg)
Photo by <a href="https://unsplash.com/@markusspiske?utm_source=unsplash&utm_medium=referral&utm_content=creditCopyText">Markus Spiske</a> on <a href="https://unsplash.com/photos/hvSr_CVecVI?utm_source=unsplash&utm_medium=referral&utm_content=creditCopyText">Unsplash</a>
  

### The Problem
Pandas dataframes come with a wide range of in-built functions, but there are always new ways that Python programmers will want to interrogate and transform their data.

### The Opportunity
A common approach is to write functions and pass the dataframes as parameters but this can become unwieldly whereas the optimum approach would be to add new methods and properties directly to the pandas classes.

### The Way Forward
It is very easy to extend the functionality classes in pandas or any other library with custom methods and properties producing code which is easy to read, understand and re-use.

## Background

I had been writing some code for a causal inference library that involved some complex data operations and the constant calls out to functions made the code difficult to read and understnad.

I hit on the idea that if pandas DataFrames did the things that I was writing functions for natively the code would become clean, simple and easy to understand and it was only a small jump from there to finding out how to extend pandas myself.

## Solution

The first function I wanted to implement was probability and conditional probability. Let us consider the following dataset ...

In [2]:
import pandas as pd
import re
from icecream import ic

import sys
sys.path.insert(1, r'C:\\Users\\GHarr\\OneDrive - Lincoln College\\Python Projects\Data Science\\ghlibrary')

# pylint: disable=import-error, unused-import
from synthetic_tools import SampleBinaryDatasets

ic.disable()

In [3]:
df_training = SampleBinaryDatasets(population_size=800, abbreviated=False).training_dataset().data
df_training

,training,skills,income
0,1,1,1
1,0,0,1
2,1,1,1
3,1,1,1
4,0,0,0
...,...,...,...
795,1,1,1
796,1,1,0
797,0,0,0
798,1,0,1


This is a synthetic dataset that captures the relationship between training, skills and income with each row representing an individual who either trained (or did not), acquired skills (or did not) and increased their income (or did not).

It would be very useful to know - given our data what is the probability that training was received. This can be expressed as a mathematical formula as follows ...

$$
\mathbb{P}(training=1)
$$

... which means "the probability that training = 1".

The traditional way of working this out would be to write specific code to filter the ``DataFrame`` down to just the rows where ``training==`` and count the rows in the filter as a proportion of the overall data ...

In [9]:
training_filter = df_training["training"]==1
df_training[training_filter].shape[0] / df_training.shape[0]

0.5

This is fairly straightforward but specific code has to be written every time and it is not immediately obvious that this code related to a probability. Wouldn't it be much better if we could simple write ``df_training.probability("p(training=1)")``?

Well, it turns out that this can be done in just 6 lines of code for any probability or conditional probability!

In [53]:
VARIABLE_REGEX: str = r'[a-zA-Z0-9_ "\']+'
OPERATOR_REGEX: str = r'=|<|<=|>|>=|!='
VALUE_REGEX: str = r'\'[^\']*\'|\"[^\"]*\"|-?\d+\.?\d*'
ASSIGNMENT_REGEX: str = fr'\s*({VARIABLE_REGEX})\s*({OPERATOR_REGEX})\s*({VALUE_REGEX})'
PROB_REGEX: str = r'(?i)^p\(([^|]+)(?:\s*\|\s*)?(.*)?\)$'

def _filter_dataset(data: pd.DataFrame, expression: str) -> pd.DataFrame:
    filter_expr = pd.Series(True, index=data.index) if expression == "" else eval(" & ".join([f"(data['{filter[0].strip()}'] {'==' if filter[1] == '=' else filter[1]} {filter[2]})" for filter in re.findall(ASSIGNMENT_REGEX, expression)]))
    return data[filter_expr]

def probability(self: pd.DataFrame, expression: str) -> float:
    parse_probability_expr = re.findall(PROB_REGEX, expression) # Parses "p(A=5, B=6)" into [('A=5, B=6', '')] or "p(A=5, B=6 | X=1, Y=2)" into [('A=5, B=6 ', 'X=1, Y=2')]

    df_events = _filter_dataset(self, parse_probability_expr[0][1].strip()) # Calculate a filtered dataset based on the events i.e. after the "|" (or the whole dataset if there is nothing conditional i.e. no "|")
    df_outcome = _filter_dataset(df_events, parse_probability_expr[0][0].strip()) # Calculate a filtered dataset based on the outcome

    return 0 if len(df_events) == 0 else df_outcome.shape[0] / df_events.shape[0] # The probability is then simply the number of rows in the outcome dataset divided by the number of rows in the events dataset

pd.DataFrame.probability = probability # Assigns the extension method to pd.DataFrame so it can be invoked just like any other native DataFrame method


In [46]:
expression = "training=1"

re.findall(ASSIGNMENT_REGEX, expression)

[('training', '=', '1')]

In [47]:
data = df_training
filter_expr = pd.Series(True, index=data.index) if expression == "" else eval(" & ".join([f"(data['{filter[0].strip()}'] {'==' if filter[1] == '=' else filter[1]} {filter[2]})" for filter in re.findall(ASSIGNMENT_REGEX, expression)]))
filter_expr

0       True
1      False
2       True
3       True
4      False
       ...  
795     True
796     True
797    False
798     True
799    False
Name: training, Length: 800, dtype: bool

In [48]:
expression = "p(skills=1 | training=0)"
parse_probability_expr = re.findall(PROB_REGEX, expression)

parse_probability_expr

[('skills=1 ', 'training=0')]

In [55]:
parse_probability_expr[0][1].strip()

'training=0'

In [57]:
df_events = _filter_dataset(df_training, parse_probability_expr[0][1].strip())
df_events.head()

,training,skills,income
1,0,0,1
4,0,0,0
5,0,0,0
8,0,0,0
12,0,0,0


In [56]:
parse_probability_expr[0][0].strip()

'skills=1'

In [58]:
df_outcome = _filter_dataset(df_events, parse_probability_expr[0][0].strip())

In [45]:
data[filter_expr]

,training,skills,income
0,1,1,1
2,1,1,1
3,1,1,1
6,1,1,1
7,1,1,1
...,...,...,...
791,1,1,1
794,1,1,0
795,1,1,1
796,1,1,0


In [54]:
df_training.probability("p(training=1)")

0.5

In [37]:
df_training.probability("p(skills=1 | training=0)")

0.05

In [6]:
df_training.probability("p(skills=1 | training=1)")

0.95

In [7]:
df_training.probability("p(income=1 | training=1)")

0.9025

In [8]:
df_training.probability("p(income=1 | training=0)")

0.1475

In [21]:
df_training.probability("p(income=1 | training=0, skills=1)")

0.1

In [32]:
EXPECT_REGEX: str = r'(?i)^e\[([^|]+)(?:\s*\|\s*)?(.*)?\]$'

def expected(self: pd.DataFrame, expression: str) -> float:
    """Calculates the expected value or conditional expected value of a DataFrame given a string expression

        Args:
            expression (str): The string expression

        Returns:
            float: The expected value

        Examples:
            >>> df.expected_value("E[D]")
            >>> 4.412406593060421

            >>> df.expected("E[D| G >= 3, E > 2]")
            >>> 1.9753517006699204
        """
    parse_expected_expr = re.findall(EXPECT_REGEX, expression)
    df_events = _filter_dataset(self, parse_expected_expr[0][1].strip()) # Calculate a filtered dataset based on the events i.e. after the "|" (or the whole dataset if there is nothing conditional i.e. no "|")

    return 0 if len(df_events) == 0 else df_events[parse_expected_expr[0][0].strip()].mean() # Calculate the expected value based on the filtered dataset

pd.DataFrame.expected = expected

In [10]:
from dag_tools import DirectedAcyclicGraph
from causal_tools import CausalModel

In [18]:
example_dag = DirectedAcyclicGraph.example_dags()["book_of_why_smoking_asthma_2"]

causal_model = CausalModel(dag=example_dag["dag"], random_state=42)
causal_model.fit(X = None)

df_continuous = causal_model.data
df_continuous

,D,F,G,A,C,B,E,X,Y
0,5.634316,1.399732,2.035583,9.914077,-24.214419,44.195973,-75.865578,-342.413180,-604.177195
1,7.341857,0.031668,4.376354,15.308372,-22.692390,29.534663,-68.266770,-199.271975,-613.181989
2,3.914095,2.636184,4.140724,6.044142,-15.317655,21.067061,-48.658849,-163.918270,-470.942156
3,3.914127,2.046147,3.231601,8.811971,-16.196070,22.381185,-47.754029,-185.314511,-545.992623
4,7.451454,3.017559,4.871805,15.313706,-31.482503,45.578518,-91.170220,-405.028262,-1026.669754
...,...,...,...,...,...,...,...,...,...
995,5.620962,-0.423221,3.966892,9.614154,-14.551332,17.128210,-39.285287,-153.175943,-493.048433
996,3.256654,0.968543,2.019814,7.040034,-12.401946,9.879755,-44.832499,-160.515481,-326.960258
997,5.487806,-0.158014,3.183499,10.789441,-15.028534,17.452708,-46.887088,-143.251753,-410.673777
998,7.100604,-0.023013,3.679870,14.205608,-22.450353,21.982254,-58.815147,-215.682315,-571.397425


In [25]:
df_continuous.expected("E[D]")

4.412406593060421

In [38]:
df_continuous.expected("E[D | G>3]")

4.429821998698432

In [39]:
df_continuous.expected("E[D| G>3, A <=12]")

3.7051574107849214